In [14]:
# allow to import modules from the project root directory
import sys
import os

# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [15]:
import great_expectations as gx

In [16]:
# Create Great Expectations context and load data
from data_quality import load_context_and_sample_data


context, da = load_context_and_sample_data("../services", "../data/samples/sample.csv")
batch_request = da.build_batch_request()

FileDataContext loading fluent config
Loading 'datasources' ->
[{'assets': [...], 'name': 'sample_data', 'type': 'pandas'}]
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
Loaded 'sample_data' from fluent config
Saving 1 Fluent Datasources to /home/sv-cheats-1/Documents/PROJECTS/MLops-project/notebooks/../services/gx/great_expectations.yml
PandasDatasource.dict() - missing config_provider, skipping config substitution
Saving 1 Fluent Datasources to /home/sv-cheats-1/Documents/PROJECTS/MLops-project/notebooks/../services/gx/great_expectations.yml
CSVAsset.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution


In [17]:
context

{
  "anonymous_usage_statistics": {
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "explicit_url": false,
    "enabled": true,
    "data_context_id": "15787a4b-eb43-4f8e-ba28-9add2ef7a91a",
    "explicit_id": true
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {},
  "evaluation_parameter_store_name": "evaluation_parameter_store",
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {
    "sample_data": {
      "type": "pandas",
      "ass

In [18]:
validator = context.get_validator(
    batch_request=batch_request,
)
# validator.head()

In [20]:
from pandas import read_csv
from src.data import preprocess_data


df = read_csv(
    context.get_datasource("sample_data").get_asset("sample_file").filepath_or_buffer
)
X, y = preprocess_data(df)

In [21]:
ds = context.sources.add_or_update_pandas(name="transformed_sample")

Saving 2 Fluent Datasources to /home/sv-cheats-1/Documents/PROJECTS/MLops-project/notebooks/../services/gx/great_expectations.yml
CSVAsset.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution


In [22]:
ds_X = ds.add_dataframe_asset(name="X", dataframe=X)
ds_y = ds.add_dataframe_asset(name="y", dataframe=y)

Saving 2 Fluent Datasources to /home/sv-cheats-1/Documents/PROJECTS/MLops-project/notebooks/../services/gx/great_expectations.yml
CSVAsset.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution
DataFrameAsset.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution
Saving 2 Fluent Datasources to /home/sv-cheats-1/Documents/PROJECTS/MLops-project/notebooks/../services/gx/great_expectations.yml
CSVAsset.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution
DataFrameAsset.dict() - missing config_provider, skipping config substitution
DataFrameAsset.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution


In [25]:
data_asset = context.get_datasource("transformed_sample").get_asset("X")
batch_request = data_asset.build_batch_request()

In [26]:
context.add_or_update_expectation_suite("feature_transform")

{
  "expectation_suite_name": "feature_transform",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.18"
  }
}

In [28]:
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="feature_transform",
)
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Airline,Origin,Dest,Distance,DayOfWeek,DOT_ID_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,Flight_Number_Operating_Airline,OriginAirportID,...,CRSElapsedTimeHH_sin,CRSElapsedTimeHH_cos,CRSElapsedTimeMM_sin,CRSElapsedTimeMM_cos,Month_sin,Month_cos,DayofMonth_sin,DayofMonth_cos,Quarter_sin,Quarter_cos
0,561,43,277,472.0,1,19977,2655,19977,2655,13871,...,0.000000,1.000000,-0.743145,-0.669131,1.224647e-16,-1.0,-0.790776,-0.612106,1.224647e-16,-1.0
1,561,895,62,1969.0,1,19977,2654,19977,2654,10693,...,0.500000,0.866025,0.500000,-0.866025,1.224647e-16,-1.0,-0.790776,-0.612106,1.224647e-16,-1.0
2,561,424,241,925.0,1,19977,2653,19977,2653,12266,...,0.258819,0.965926,0.587785,0.809017,1.224647e-16,-1.0,-0.790776,-0.612106,1.224647e-16,-1.0
3,561,33,62,337.0,1,19977,2652,19977,2652,12892,...,0.000000,1.000000,-0.809017,-0.587785,1.224647e-16,-1.0,-0.790776,-0.612106,1.224647e-16,-1.0
4,561,62,43,2704.0,1,19977,2652,19977,2652,14771,...,0.707107,0.707107,-0.994522,0.104528,1.224647e-16,-1.0,-0.790776,-0.612106,1.224647e-16,-1.0


### 1. Check that no null values exist

In [ ]:
for column in validator.columns():
    validator.expect_column_values_to_not_be_null(column=column)

### 2. Check that columns that were hashed are hashed properly

In [ ]:
from omegaconf import DictConfig, OmegaConf
from hydra import initialize, compose

with initialize(version_base=None, config_path="../configs"):
    cfg = compose(config_name="data_transformations")
    for column in cfg["hash_features"]:
        validator.expect_column_values_to_be_between(
            column=column,
            min_value=0,
            max_value=999,
        )

### 3. Check that datatypes are correct (everything is a number) 

In [ ]:
for column in validator.columns():
    validator.expect_column_values_to_be_of_type(column=column, type_=str(X[column].dtype))

### 4. Check that cyclic data in encoded correctly

In [ ]:
with initialize(version_base=None, config_path="../configs"):
    for column in filter(lambda c: c.endswith('cos') or c.endswith('sin'), validator.columns()):
        validator.expect_column_values_to_be_between(
            column=column,
            min_value=-1,
            max_value=1,
        )

In [53]:
print("Total number of expectations:", len(validator.expectation_suite.expectations))

Total number of expectations: 87


In [54]:
# Save expectations and checkpoint
validator.save_expectation_suite(discard_failed_expectations=False)
checkpoint = context.add_or_update_checkpoint(
    name="transformations_checkpoint",
    validator=validator,
)

	87 expectation(s) included in expectation_suite.


In [55]:
checkpoint_result = checkpoint.run()

# Open report in browser
context.view_validation_result(checkpoint_result)

if checkpoint_result.success:
    print("Data quality verification passed successfully")
else:
    print("Data quality verification failed")

	87 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/233 [00:00<?, ?it/s]

Data quality verification passed successfully
